# mBuild Tutorial 09: Surface Functionalization

This tutorial demonstrates the use of the surface functionalization routines included in mBuild. 

As in the prior tutorials, we need to first import mbuild (here as `mb`).

In [ ]:
import mbuild as mb

Here, we will examine the functionalization of surfaces with alkane chains.  Since we will make use of `Polymer` class, we'll need to first define the CH2 moiety, as before.

In [ ]:
class CH2(mb.Compound):
    def __init__(self):
        super(CH2, self).__init__()
        
        mb.load('ch2.pdb', compound=self)
        carbon = list(self.particles_by_name('C'))[0]
        up_port = mb.Port(anchor=carbon, orientation=[0, 0, 1], separation=0.075)
        down_port = mb.Port(anchor=carbon, orientation=[0, 0, -1], separation=0.075)
        self.add(up_port, label='up')
        self.add(down_port, label='down')

mBuild features several functions to aid in the functionalization of surfaces. For example, the `Pattern.apply_to_compound` method allows one to connect copies of a 'guest' `Compound` to `Ports` located on a 'host' `Compound`. We'll explore how this can be useful for surface functionalization by considering a crystalline silica surface (featuring many `Ports`) as our host and a polymer chain as our guest.

First we'll import our crystalline silica surface from mBuild's `surfaces` library.  Note, this structure has surface binding sites already identified, with `Ports` placed at these locations. In the future we hope to add routines to mBuild to automatically detect surface sites and add `Ports` to them.

In [ ]:
from mbuild.lib.surfaces import Betacristobalite
surface = Betacristobalite()
surface.visualize(show_ports=True)

mBuild also features a recipe that creates a silica interface by carving from bulk amorphous silica.

In [ ]:
from mbuild.lib.bulk_materials import AmorphousSilica
from mbuild.recipes import SilicaInterface

surface = SilicaInterface(bulk_silica=AmorphousSilica())
surface.visualize(show_ports=True)

Now, we will  create the monolayer surface, via the following steps:
- create prototypes for two polymer chains of different lengths
- specify a random pattern of 30 points in 2D space
- use `apply_to_compound` to stick copies of the first polymer on the surface, backfilling unused `Ports` with the shorter polymer 

In the mBuild nomenclature, `guests` are the `Compound` copies that have been added to the surface and `backfills` are an optional second `Compound` type that can be used to fill any leftover `Ports` in the host `Compound` after all points in the `Pattern` have been satisfied.

Note, these chains are uncapped, but could be easily capped following the same procedures in the prior tutorials. 

In [ ]:
surface = Betacristobalite()

alkane_12 = mb.Polymer(monomers=CH2(), n=12, port_labels=('up', 'down'))
alkane_3 = mb.Polymer(monomers=CH2(), n=3, port_labels=('up', 'down'))
pattern = mb.Grid2DPattern(5, 5)
guests, backfills = pattern.apply_to_compound(guest=alkane_12, host=surface, backfill=alkane_3, backfill_port_name='down')
functionalized_surface = mb.Compound(subcompounds=[surface, guests, backfills])
functionalized_surface.visualize()

As we've seen, the `Pattern.apply_to_compound` method is a useful way to approach surface functionalization with mBuild. However, this can be done even easier by using `mbuild.Monolayer`, where the above steps have been wrapped into a class. Multi-component monolayers can be generated by simply passing a list of `Compounds` to the `chains` argument also with the `fractions` of each component.

In [ ]:
surface = SilicaInterface(bulk_silica=AmorphousSilica())
alkane_18 = mb.Polymer(monomers=CH2(), n=18, port_labels=('up', 'down'))
alkane_12 = mb.Polymer(monomers=CH2(), n=12, port_labels=('up', 'down'))
alkane_3 = mb.Polymer(monomers=CH2(), n=3, port_labels=('up', 'down'))

monolayer = mb.Monolayer(surface=surface, chains=(alkane_3, alkane_12, alkane_18), fractions=(0.5, 0.4, 0.1))
monolayer.visualize()

So far, the examples we have looked at have used surfaces already contained within mBuild that have `Ports` identified. However, while the surfaces available within mBuild are currently limited to crystalline and amorphous silica, additional surfaces can easily be created. For example, consider this example, where mBuild's `Lattice` class is utilized to generate a gold surface, which we will then functionalize with alkanethiols.

First, we'll use `mb.Lattice` to create an face-centered cubic gold surface.

In [ ]:
gold = mb.Compound(name='Au')
lattice_vectors = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
spacing = [.40782, .40782, .40782]
locations = [[0., 0., 0.], [.5, .5, 0.],
             [.5, 0., .5], [0., .5, .5]]
basis = {'Au': locations}
lattice = mb.Lattice(lattice_spacing=spacing,
                     lattice_vectors=lattice_vectors,
                     lattice_points=basis)
fcc_gold = lattice.populate(x=6, y=6, z=2,
                            compound_dict={'Au': gold})
fcc_gold.visualize()

Now, we'll identify the atoms at the surface and attach ports to them. To do this, we'll simply consider all atoms within 0.05nm of the atom with the largest *z* coordinate to be at the surface.

In [ ]:
gold_with_ports = mb.Compound()
gold_with_ports.add(fcc_gold)

buffer = 0.05
surface_height = max(fcc_gold.xyz[:,2])
surface_atoms = [atom for atom in fcc_gold if atom.pos[2] >= surface_height - buffer]
for atom in surface_atoms:
    port = mb.Port(anchor=atom, orientation=[0, 0, 1], separation=0.12)
    gold_with_ports.add(port)

gold_with_ports.visualize(show_ports=True)

We'll now define a class for an alkanethiol chain. For this we will import the `Alkane` class from mBuild as well as define a class for a sulfur headgroup.

In [ ]:
from mbuild.examples import Alkane

class S(mb.Compound):
    def __init__(self):
        super(S, self).__init__()
        
        self.add(mb.Particle(name='S'))
        up_port = mb.Port(anchor=self[0], orientation=[0, 0, 1], separation=0.1)
        self.add(up_port, 'up')
        down_port = mb.Port(anchor=self[0], orientation=[0, 0, -1], separation=0.1)
        self.add(down_port, 'down')

class Alkanethiol(mb.Compound):
    def __init__(self, n):
        super(Alkanethiol, self).__init__()
        
        self.add(S(), 'headgroup')
        chain = Alkane(n, cap_end=False)
        mb.force_overlap(move_this=chain,
                         from_positions=chain['down'],
                         to_positions=self['headgroup']['up'])
        self.add(chain)
        self.add(self['headgroup']['down'], 'down', containment=False)

We can now create a butanethiol by instantiating the `Alkanethiol` class with a chain length of 4.

In [ ]:
butanethiol = Alkanethiol(4)
butanethiol.visualize(show_ports=True)

To attach copies of the butanethiol to the surface, we simply provide a call to `mb.Monolayer`.

In [ ]:
monolayer = mb.Monolayer(surface=gold_with_ports, chains=butanethiol)
monolayer.visualize()

## Recap

The goal of this tutorial was to demonstrate how to functionalize surfaces using mBuild.